# APIs Modules - ein erste Blick auf "Requests"

Gezielt und automatisiert Daten im Web auslesen

Zuerst installieren wir pandas und requests. Das können wir von der Command line mit den Befehlen ```pip install requests``` und ```pip install pandas``` die beiden Module installieren. Oder direkt hier in Jupyter Notebook mit ```!pip install requests``` und ```!pip install pandas```.

### Unsere Imports

In [90]:
import requests
import pandas as pd
import calendar, datetime, time

Zur Dokumentation von requestst geht es [hier lang](http://docs.python-requests.org/en/master/).

### Notes

In einer ersten Übung arbeiten wir mit [der API des USGS](https://earthquake.usgs.gov/fdsnws/event/1/#methods) arbeiten. Für die meisten API muss man sich heutezutage eine eigene API-Key kreieren. Das werden wir zu einem späteren Zeitpunkt tun. Diese API braucht keine Keys. Deshalb ist es ein gutes Kennenlernbeispiel.

Falls Probleme mit dem Output auftauchen, hier ein Work around für die Meldung, dass der Output limitiert wurde. [Reconfigure line output](https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-when-viewing-image-in-jupyter-notebook) and [opening stuff from the command line](https://stackoverflow.com/questions/16344709/how-to-open-a-file-from-the-command-line-with-a-specified-program)

### Dokumentation 

Bevor man mit deiner API arbeitet, lohnt es sich immer zuerst, die [Dokumentation](https://earthquake.usgs.gov/fdsnws/event/1/#parameters) anzuschauen. 

### Erste Abfragen

Kleinere Abfrage. Wieviele Erdbeben hat es in einem bestimmten Zeitraum gegeben?

In [18]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20&endtime=2017-09-21'

In [19]:
response = requests.get(url)

In [20]:
response

<Response [200]>

In [22]:
response.text

'238'

Wieviele hat es letzte Nacht von 22 bis 6 Uhr morgens gegeben? Dafür müssen wir in der Dokumentation Time nachfragen.

In [30]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20T22:00:00&endtime=2017-09-21T06:00:00'

Wegen Zeitverschiebung Zeit anpassen.

In [33]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20T24:00:00&endtime=2017-09-21T08:00:00'

In [35]:
response = requests.get(url)
response.text

'31'

### Grosse, komibinierte Abfrage

Details zu allen Erbeben in den über drei Tage, 17-9-2017 bis 21-9-2017.

In [55]:
url1 = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'

In [56]:
urlzeit = '&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00'

In [52]:
response = requests.get(url1+urlzeit)
response.text

'{"type":"FeatureCollection","metadata":{"generated":1505979266000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00","title":"USGS Earthquakes","status":200,"api":"1.5.8","count":777},"features":[{"type":"Feature","properties":{"mag":1.42,"place":"1km ESE of Pacheco, California","time":1505978530260,"updated":1505979122528,"tz":-480,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/nc72896141","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc72896141&format=geojson","felt":null,"cdi":null,"mmi":null,"alert":null,"status":"automatic","tsunami":0,"sig":31,"net":"nc","code":"72896141","ids":",nc72896141,","sources":",nc,","types":",geoserve,nearby-cities,origin,phase-data,scitech-link,","nst":12,"dmin":0.05685,"rms":0.09,"gap":92,"magType":"md","type":"earthquake","title":"M 1.4 - 1km ESE of Pacheco, California"},"geometry":{"type":"Point","coordinates":[-122.0561676,37.9796677,13.4]

Alle Erbeben mit den ungefähren Koordinaten von Mexiko. [Hier könnte man die ungefähr nachschlagen.](http://www.latlong.net/)

In [57]:
urlloc = '&minlatitude=13&maxlatitude=33&minlongitude=-120&maxlongitude=-85'

In [58]:
response = requests.get(url1+urlzeit+urlloc)
response.text

'{"type":"FeatureCollection","metadata":{"generated":1505979270000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00&minlatitude=13&maxlatitude=33&minlongitude=-120&maxlongitude=-85","title":"USGS Earthquakes","status":200,"api":"1.5.8","count":32},"features":[{"type":"Feature","properties":{"mag":2.26,"place":"4km SW of Imperial, CA","time":1505947525530,"updated":1505948918700,"tz":-480,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson","felt":null,"cdi":null,"mmi":null,"alert":null,"status":"reviewed","tsunami":0,"sig":79,"net":"ci","code":"38007144","ids":",ci38007144,","sources":",ci,","types":",focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,","nst":41,"dmin":0.08931,"rms":0.26,"gap":102,"magType":"ml","type":"earthquake","title":"M 2.3 - 4km SW of Imperial, CA"},"geomet

Dieser Wulst lässt sich aber ziemlich schwer lesen. Deshalb müssen wir das zuerst umformatieren und dann verstehen, wie das ganze strukturiert ist. 

In [59]:
response.json()

{'bbox': [-117.0765, 14.5296, -0.2, -90.9069, 32.9196667, 177.24],
 'features': [{'geometry': {'coordinates': [-115.6006667, 32.8185, 11.77],
    'type': 'Point'},
   'id': 'ci38007144',
   'properties': {'alert': None,
    'cdi': None,
    'code': '38007144',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson',
    'dmin': 0.08931,
    'felt': None,
    'gap': 102,
    'ids': ',ci38007144,',
    'mag': 2.26,
    'magType': 'ml',
    'mmi': None,
    'net': 'ci',
    'nst': 41,
    'place': '4km SW of Imperial, CA',
    'rms': 0.26,
    'sig': 79,
    'sources': ',ci,',
    'status': 'reviewed',
    'time': 1505947525530,
    'title': 'M 2.3 - 4km SW of Imperial, CA',
    'tsunami': 0,
    'type': 'earthquake',
    'types': ',focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,',
    'tz': -480,
    'updated': 1505948918700,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144'},
   'type': 'Feature'},

In [60]:
dct = response.json()

Um sicher zu gehen, prüfen wir nochmals den Datentyp.

In [62]:
type(dct)

dict

Schauen wir uns alle keys und values an

In [72]:
for key, value in dct.items() :
    print (key, value)

type FeatureCollection
bbox [-117.0765, 14.5296, -0.2, -90.9069, 32.9196667, 177.24]
features [{'id': 'ci38007144', 'type': 'Feature', 'properties': {'updated': 1505948918700, 'sig': 79, 'sources': ',ci,', 'tz': -480, 'type': 'earthquake', 'alert': None, 'types': ',focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,', 'mmi': None, 'dmin': 0.08931, 'magType': 'ml', 'status': 'reviewed', 'time': 1505947525530, 'code': '38007144', 'net': 'ci', 'rms': 0.26, 'title': 'M 2.3 - 4km SW of Imperial, CA', 'cdi': None, 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson', 'mag': 2.26, 'tsunami': 0, 'place': '4km SW of Imperial, CA', 'felt': None, 'gap': 102, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144', 'nst': 41, 'ids': ',ci38007144,'}, 'geometry': {'type': 'Point', 'coordinates': [-115.6006667, 32.8185, 11.77]}}, {'id': 'ci38007104', 'type': 'Feature', 'properties': {'updated': 1505938469023, 'sig': 15, 'sourc

Features interessiert uns.

In [74]:
len(dct['features'])

32

Schauen wir uns das erste Element an

In [80]:
dct['features'][0]['properties']

{'alert': None,
 'cdi': None,
 'code': '38007144',
 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson',
 'dmin': 0.08931,
 'felt': None,
 'gap': 102,
 'ids': ',ci38007144,',
 'mag': 2.26,
 'magType': 'ml',
 'mmi': None,
 'net': 'ci',
 'nst': 41,
 'place': '4km SW of Imperial, CA',
 'rms': 0.26,
 'sig': 79,
 'sources': ',ci,',
 'status': 'reviewed',
 'time': 1505947525530,
 'title': 'M 2.3 - 4km SW of Imperial, CA',
 'tsunami': 0,
 'type': 'earthquake',
 'types': ',focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,',
 'tz': -480,
 'updated': 1505948918700,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144'}

Lesen wir folgendes aus:
1. Type
2. Magnitude
4. Örtlichkeit
5. Time 

In [81]:
earthquakes = []

for elem in dct['features']:
    t = elem['properties']['type']
    m = elem['properties']['mag']
    p = elem['properties']['place']
    tm = elem['properties']['time']
    
    mini_dict = {'Type': t,
                 'Mag': m,
                 'Place': p,
                 'Time': tm}
    
    earthquakes.append(mini_dict)

Dealing with Unix time

In [116]:
earthquakes

[{'Mag': 2.26,
  'Place': '4km SW of Imperial, CA',
  'Time': datetime.datetime(2017, 9, 21, 0, 45, 25),
  'Type': 'earthquake'},
 {'Mag': 1,
  'Place': '7km NW of La Mesa, CA',
  'Time': datetime.datetime(2017, 9, 20, 22, 0, 49),
  'Type': 'quarry blast'},
 {'Mag': 2.71,
  'Place': '19km W of Delta, B.C., MX',
  'Time': datetime.datetime(2017, 9, 20, 20, 22, 27),
  'Type': 'earthquake'},
 {'Mag': 0.94,
  'Place': '27km SSW of Ocotillo Wells, CA',
  'Time': datetime.datetime(2017, 9, 20, 17, 47, 3),
  'Type': 'earthquake'},
 {'Mag': 4.7,
  'Place': '51km SSW of Pijijiapan, Mexico',
  'Time': datetime.datetime(2017, 9, 20, 12, 39, 28),
  'Type': 'earthquake'},
 {'Mag': 4.6,
  'Place': '15km ESE of San Mateo del Mar, Mexico',
  'Time': datetime.datetime(2017, 9, 20, 5, 28, 5),
  'Type': 'earthquake'},
 {'Mag': 4.9,
  'Place': '4km ESE of Acatenango, Guatemala',
  'Time': datetime.datetime(2017, 9, 20, 4, 37, 35),
  'Type': 'earthquake'},
 {'Mag': 4.6,
  'Place': '15km SW of San Dionisio 

In [114]:
earthquakes = []

for elem in dct['features']:
    t = elem['properties']['type']
    m = elem['properties']['mag']
    p = elem['properties']['place']
    
    #Ints bearbeiten, die letzten drei Stellen abschneiden
    tm = dt(int(str(elem['properties']['time'])[:-3])) 
    
    mini_dict = {'Type': t,
                 'Mag': m,
                 'Place': p,
                 'Time': tm}
    
    earthquakes.append(mini_dict)

In [115]:
earthquakes

[{'Mag': 2.26,
  'Place': '4km SW of Imperial, CA',
  'Time': datetime.datetime(2017, 9, 21, 0, 45, 25),
  'Type': 'earthquake'},
 {'Mag': 1,
  'Place': '7km NW of La Mesa, CA',
  'Time': datetime.datetime(2017, 9, 20, 22, 0, 49),
  'Type': 'quarry blast'},
 {'Mag': 2.71,
  'Place': '19km W of Delta, B.C., MX',
  'Time': datetime.datetime(2017, 9, 20, 20, 22, 27),
  'Type': 'earthquake'},
 {'Mag': 0.94,
  'Place': '27km SSW of Ocotillo Wells, CA',
  'Time': datetime.datetime(2017, 9, 20, 17, 47, 3),
  'Type': 'earthquake'},
 {'Mag': 4.7,
  'Place': '51km SSW of Pijijiapan, Mexico',
  'Time': datetime.datetime(2017, 9, 20, 12, 39, 28),
  'Type': 'earthquake'},
 {'Mag': 4.6,
  'Place': '15km ESE of San Mateo del Mar, Mexico',
  'Time': datetime.datetime(2017, 9, 20, 5, 28, 5),
  'Type': 'earthquake'},
 {'Mag': 4.9,
  'Place': '4km ESE of Acatenango, Guatemala',
  'Time': datetime.datetime(2017, 9, 20, 4, 37, 35),
  'Type': 'earthquake'},
 {'Mag': 4.6,
  'Place': '15km SW of San Dionisio 

### Erste Erfahrung mit Pandas

In [118]:
pd.DataFrame(earthquakes)

,Mag,Place,Time,Type
0,2.26,"4km SW of Imperial, CA",2017-09-21 00:45:25,earthquake
1,1.00,"7km NW of La Mesa, CA",2017-09-20 22:00:49,quarry blast
2,2.71,"19km W of Delta, B.C., MX",2017-09-20 20:22:27,earthquake
3,0.94,"27km SSW of Ocotillo Wells, CA",2017-09-20 17:47:03,earthquake
4,4.70,"51km SSW of Pijijiapan, Mexico",2017-09-20 12:39:28,earthquake
5,4.60,"15km ESE of San Mateo del Mar, Mexico",2017-09-20 05:28:05,earthquake
6,4.90,"4km ESE of Acatenango, Guatemala",2017-09-20 04:37:35,earthquake
7,4.60,"15km SW of San Dionisio del Mar, Mexico",2017-09-20 01:11:34,earthquake
8,4.90,"101km S of San Francisco del Mar, Mexico",2017-09-19 23:13:19,earthquake
9,7.10,"5km ENE of Raboso, Mexico",2017-09-19 20:14:39,earthquake


In [119]:
df = pd.DataFrame(earthquakes)

In [122]:
df.sort_values(by='Mag', ascending=False)

,Mag,Place,Time,Type
9,7.10,"5km ENE of Raboso, Mexico",2017-09-19 20:14:39,earthquake
28,5.60,"108km S of San Francisco del Mar, Mexico",2017-09-18 16:19:12,earthquake
11,5.40,"98km SSW of Tres Picos, Mexico",2017-09-19 09:53:17,earthquake
12,4.90,"59km SSE of San Francisco del Mar, Mexico",2017-09-19 09:03:02,earthquake
24,4.90,"101km S of San Francisco del Mar, Mexico",2017-09-18 21:51:32,earthquake
29,4.90,"6km SW of San Mateo del Mar, Mexico",2017-09-18 14:07:00,earthquake
6,4.90,"4km ESE of Acatenango, Guatemala",2017-09-20 04:37:35,earthquake
8,4.90,"101km S of San Francisco del Mar, Mexico",2017-09-19 23:13:19,earthquake
4,4.70,"51km SSW of Pijijiapan, Mexico",2017-09-20 12:39:28,earthquake
27,4.70,"116km S of San Francisco del Mar, Mexico",2017-09-18 17:13:59,earthquake
